Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747540875350, experiment_id='0', last_update_time=1747540875350, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/17 22:04:49 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/05/17 22:04:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/fb2a3087ff1b42d9a3f8e4f9b12b500d/artifacts'
🏃 View run dapper-slug-575 at: http://127.0.0.1:5000/#/experiments/1/runs/fb2a3087ff1b42d9a3f8e4f9b12b500d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [5]:
# mlflow creates a random id for the new experiment unlike in db system where the ID are sequential
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1747541089560, experiment_id='1', last_update_time=1747541089560, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747540875350, experiment_id='0', last_update_time=1747540875350, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()

[]

In [8]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids=["1"],
    filter_string="",
    order_by=["metrics.rmse ASC"],
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5
)
print(runs)

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "fb2a3087ff1b42d9a3f8e4f9b12b500d", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2025-05-18 04:04:50.795737", "model_uuid": '
                             '"47591c87b85c4f35953243e95d576ea2", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.7", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.1", '
                             '"serialization_format": "cloudpickle", "code": '
             

In [10]:
first_run_id = runs[0].info.run_id

In [11]:
mlflow.register_model(
    model_uri=f'runs:/{first_run_id}/models',
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/17 22:16:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1747541799102, current_stage='None', description='', last_updated_timestamp=1747541799102, name='iris-classifier', run_id='fb2a3087ff1b42d9a3f8e4f9b12b500d', run_link='', source='mlflow-artifacts:/1/fb2a3087ff1b42d9a3f8e4f9b12b500d/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>